In [1]:
# This is necessary to recognize the modules
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

import asyncio
from decimal import Decimal
from core.data_sources import CLOBDataSource
from core.features.candles.volatility import VolatilityConfig
from research_notebooks.xtreet_bb.utils import generate_config, dump_dict_to_yaml

clob = CLOBDataSource()

In [3]:
# Screener parameters
CONNECTOR_NAME = "binance_perpetual"
INTERVALS = ["1m"]
DAYS = 7
FETCH_CANDLES = False
BATCH_CANDLES_REQUEST = 15
SLEEP_REQUEST = 60.0
VOLUME_THRESHOLD = 0.3  # From percentile VOLUME_THRESHOLD to 1
VOLATILITY_THRESHOLD = 0.4  # From percentile VOLATILITY_THRESHOLD to 1

# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

VOLATILITY_WINDOW = 60  # In bars
VOLUME_FAST_WINDOW = 20  # No se usa
VOLUME_SLOW_WINDOW = 100  # No se usa

# Config generation
TOTAL_AMOUNT = 300  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_EXECUTORS_PER_SIDE = 1  # Maximum number of executors per side
COOLDOWN_TIME = 0
LEVERAGE = 20 # Should be for each trading pair
TIME_LIMIT = 60 * 60 * 24
BOLLINGER_LENGTHS = [50, 100, 200]
BOLLINGER_STDS = [1.4, 1.8, 2.2]
SL_STD_MULTIPLIER = 2
TS_DELTA_MULTIPLIER = 0.2

# Config filtering
MIN_DISTANCE_BETWEEN_ORDERS = 0.01
MAX_TS_SL_RATIO = 0.5

# Download data
- Get trading rules
- Get candles for the last x days

In [4]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
    .get_all_trading_pairs()

if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAME,
        trading_pair=trading_pair,
        interval=interval,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch for interval in INTERVALS]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))
candles = [value for key, value in clob.candles_cache.items() if key[2] in INTERVALS and key[0] == CONNECTOR_NAME]

2024-08-31 20:47:04,185 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1462e64d0>
2024-08-31 20:47:04,186 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x146231000>, 716.700306291)]']
connector: <aiohttp.connector.TCPConnector object at 0x1462e6500>


In [5]:
from core.features.candles.volume import VolumeConfig
from research_notebooks.xtreet_bb.utils import generate_screener_report

screner_report = generate_screener_report(
    candles=candles,
    trading_rules=trading_rules,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_FAST_WINDOW, long_window=VOLUME_FAST_WINDOW))
screner_report["url"] = screner_report["trading_pair"].apply(lambda x: f"https://www.binance.com/en/trade/{x}")
screner_report.sort_values("mean_natr", ascending=False, inplace=True)
screner_report

Error processing FRONT-USDT: list index out of range


,trading_pair,mean_volatility,mean_natr,mean_bb_width,average_volume_per_hour,last_price,price_step_pct,normalized_score,url
13,XEM-USDT,0.00410815,0.00559602,1.56626734,846412.32974442,0.0179,0.00558659,0.00594733,https://www.binance.com/en/trade/XEM-USDT
257,DOGS-USDT,0.00374633,0.00557404,3.85099236,58372508.26445325,0.0013144,0.00007608,1,https://www.binance.com/en/trade/DOGS-USDT
238,VIDT-USDT,0.00342757,0.00483275,3.41291894,9531744.99586725,0.05028,0.00019889,0.1446982,https://www.binance.com/en/trade/VIDT-USDT
113,SUN-USDT,0.00325588,0.00480054,3.53904681,29867774.9081979,0.036038,0.00002775,0.47005751,https://www.binance.com/en/trade/SUN-USDT
220,MBOX-USDT,0.00305356,0.00447024,2.57347104,3666078.91249911,0.1663,0.00060132,0.04201845,https://www.binance.com/en/trade/MBOX-USDT
...,...,...,...,...,...,...,...,...,...
214,LOOM-USDT,0.00070864,0.00079939,0.86434271,192434.0459994,0.04468,0.00022381,0.00073839,https://www.binance.com/en/trade/LOOM-USDT
163,XLM-USDT,0.00062574,0.00076637,0.7169193,1482130.11107517,0.09322,0.00010727,0.00471978,https://www.binance.com/en/trade/XLM-USDT
83,TRX-USDT,0.00055072,0.00074257,0.60155461,8568631.95350011,0.15937,0.00006275,0.02291065,https://www.binance.com/en/trade/TRX-USDT
33,BTCDOM-USDT,0.00033664,0.00038206,0.36554343,164958.52376029,2824.5,0.0000354,0.00026782,https://www.binance.com/en/trade/BTCDOM-USDT


In [6]:
# Calculate the 20th percentile (0.2 quantile) for both columns
natr_percentile = screner_report['mean_natr'].quantile(VOLATILITY_THRESHOLD)
volume_percentile = screner_report['average_volume_per_hour'].quantile(VOLUME_THRESHOLD)

# Filter the DataFrame to get observations where mean_natr is greater than its 20th percentile
# and average_volume_per_hour is greater than its 20th percentile
screener_top_markets = screner_report[
    (screner_report['mean_natr'] > natr_percentile) &
    (screner_report['average_volume_per_hour'] > volume_percentile) &
    (screner_report["price_step_pct"] < MAX_PRICE_STEP)
]

# Display the filtered DataFrame
screener_top_markets[["trading_pair", "mean_natr", "average_volume_per_hour", "price_step_pct", "url"]]

,trading_pair,mean_natr,average_volume_per_hour,price_step_pct,url
257,DOGS-USDT,0.00557404,58372508.26445325,0.00007608,https://www.binance.com/en/trade/DOGS-USDT
238,VIDT-USDT,0.00483275,9531744.99586725,0.00019889,https://www.binance.com/en/trade/VIDT-USDT
113,SUN-USDT,0.00480054,29867774.9081979,0.00002775,https://www.binance.com/en/trade/SUN-USDT
220,MBOX-USDT,0.00447024,3666078.91249911,0.00060132,https://www.binance.com/en/trade/MBOX-USDT
119,CHESS-USDT,0.0044066,3825026.63651922,0.00054885,https://www.binance.com/en/trade/CHESS-USDT
...,...,...,...,...,...
73,ENS-USDT,0.00162084,1688687.45631713,0.00005531,https://www.binance.com/en/trade/ENS-USDT
135,COTI-USDT,0.00161843,384330.13129969,0.00012189,https://www.binance.com/en/trade/COTI-USDT
66,AAVE-USDT,0.0016094,5634949.37285643,0.00007622,https://www.binance.com/en/trade/AAVE-USDT
43,ZEC-USDT,0.00160707,1461995.75748517,0.00030451,https://www.binance.com/en/trade/ZEC-USDT


In [7]:
strategy_configs = generate_config(
    connector_name=CONNECTOR_NAME,
    intervals=INTERVALS,
    screener_top_markets=screener_top_markets,
    candles=candles,
    total_amount=TOTAL_AMOUNT,
    max_executors_per_side=MAX_EXECUTORS_PER_SIDE,
    cooldown_time=COOLDOWN_TIME,
    leverage=LEVERAGE,
    time_limit=TIME_LIMIT,
    bb_lengths=BOLLINGER_LENGTHS,
    bb_stds=BOLLINGER_STDS,
    sl_std_multiplier=SL_STD_MULTIPLIER,
    min_distance_between_orders=MIN_DISTANCE_BETWEEN_ORDERS,
    max_ts_sl_ratio=MAX_TS_SL_RATIO,
    ts_delta_multiplier=TS_DELTA_MULTIPLIER,
)

Skipping ALT-USDT due to stop loss closer to get executed:BEP: 0.007820643823174267, SL: 0.005629139072847645, S1: 0.014109543384068517
Skipping CYBER-USDT due to stop loss closer to get executed:BEP: 0.007928974450681414, SL: 0.00527287107830214, S1: 0.013342238056086812
Skipping ALPACA-USDT due to stop loss closer to get executed:BEP: 0.011675670529016857, SL: 0.007487815684537106, S1: 0.020145253328722107
Skipping EDU-USDT due to stop loss closer to get executed:BEP: 0.0053129674825611595, SL: 0.004995318838058084, S1: 0.010519054897795302
Skipping CYBER-USDT due to stop loss closer to get executed:BEP: 0.009180902083360865, SL: 0.005859374999999974, S1: 0.015236322369992845
Skipping ETHFI-USDT due to stop loss closer to get executed:BEP: 0.0074590419806386254, SL: 0.005965697240865032, S1: 0.013840830449827002
Skipping LUNA2-USDT due to stop loss closer to get executed:BEP: 0.009466239548979516, SL: 0.006460674157303439, S1: 0.017376412701952428
Skipping USTC-USDT due to stop loss 

In [8]:
len(set([config["trading_pair"] for config in strategy_configs]))

130

In [9]:
len(strategy_configs)

2771

In [10]:
# DCA amounts
for config in strategy_configs:
    dump_dict_to_yaml("configs/", config)